# **Reporte del Modelo Baseline**
---

In [1]:
# Librerias
!pip install optuna
import os
import requests
import logging
import optuna
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.7 MB/s eta 0:00:00


In [2]:
# Funcion para crear el objeto de creación de logs
def create_logger():
  logging.basicConfig(level = logging.INFO, format = '%(asctime)s - %(levenname)s - %(message)s')
  logger = logging.getLogger('Logger')
  logger.info('Logger creado')
  return logger

In [3]:
# Función de consumo a la base de datos de Firebase para cargar el DataSet
def download_firebase(url, logger):
  logger.info("Extrayendo el archivo desde Firebase")
  df = None
  try:
    df = pd.read_csv(url)
    logger.info("Archivo cargado")
  except requests.exceptions.RequestException as e:
    logger.info(f"Error al descargar el archivo CSV: {e}")
  except pd.errors.EmptyDataError:
    logger.info("El archivo CSV está vacío.")
  except Exception as e:
    logger.info(f"Ocurrió un error inesperado: {e}")
  return df

In [4]:
# Cargar DataSet
url = 'https://firebasestorage.googleapis.com/v0/b/personalwp-8822c.appspot.com/o/diabetes_prediction_dataset.csv?alt=media&token=4d70d154-c3d0-4fa0-a3aa-9b9972dd3b95'
logger = create_logger()
df = download_firebase(url, logger)

In [5]:
# Eliminación de valores atípicos y duplicados
seventy_fifth = df['bmi'].quantile(0.75)
twenty_fifth = df['bmi'].quantile(0.25)
iqr = seventy_fifth - twenty_fifth
upper = seventy_fifth + (10 * iqr)
outliers_bmi_upper = df[(df['bmi'] > upper)]
df = pd.merge(df, outliers_bmi_upper, indicator = True, how = 'outer').query('_merge == "left_only"').drop('_merge', axis = 1)
df = df.drop_duplicates(keep = "first")

In [6]:
# Variables Categóricas a Numéricas
df['gender'] = pd.factorize(df['gender'])[0]
df['smoking_history'] = pd.factorize(df['smoking_history'])[0]

## **Implementación del Modelo**

### **Partición de Datos**

In [7]:
# Separación de la 'Data' (Características)
X = df.drop(columns = 'diabetes')
X.shape

(96143, 8)

In [8]:
# Separación del 'Target' (Variable objetivo)
y = df['diabetes']
y.shape

(96143,)

In [9]:
# Partición de los datos: 70% para entrenamiento, 30% para prueba y estratificación en las etiquetas (y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42, stratify = y)

In [10]:
# Validación de la partición de los datos
print(f'Número de muestras en entrenamiento: {X_train.shape[0]}')
print(f'Número de muestras en prueba: {X_test.shape[0]}')
print(f'Número de características: {X_train.shape[1]}')

Número de muestras en entrenamiento: 67300
Número de muestras en prueba: 28843
Número de características: 8


In [11]:
# Distribución de la variable objetivo en los conjuntos de entrenamiento y prueba
print(f'Distribución de clases en entrenamiento: {np.bincount(y_train)}')
print(f'Distribución de clases en prueba: {np.bincount(y_test)}')

Distribución de clases en entrenamiento: [61363  5937]
Distribución de clases en prueba: [26298  2545]


### **Entrenamiento del Modelo**

In [12]:
# Función para la definición y entrenamiento de hiperparámetros a explorar
def objective(trial):
  alpha = trial.suggest_float("alpha", 0.01, 10000, log=True)
  model = MultinomialNB(alpha = alpha)
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  score = accuracy_score(y_test, y_pred)
  return score

In [13]:
# Exploración de hiperparámetros
study = optuna.create_study(direction = "maximize", storage = "sqlite:///hp.db", study_name = "MultinomialNB")
study.optimize(func = objective, n_trials = 100, n_jobs = -1)

[I 2024-12-11 09:03:51,942] A new study created in RDB with name: MultinomialNB
[I 2024-12-11 09:03:52,393] Trial 0 finished with value: 0.890718718579898 and parameters: {'alpha': 28.268586218896136}. Best is trial 0 with value: 0.890718718579898.
[I 2024-12-11 09:03:52,401] Trial 1 finished with value: 0.890718718579898 and parameters: {'alpha': 0.0598218954016902}. Best is trial 0 with value: 0.890718718579898.
[I 2024-12-11 09:03:52,938] Trial 2 finished with value: 0.8924522414450646 and parameters: {'alpha': 1837.9897491203462}. Best is trial 2 with value: 0.8924522414450646.
[I 2024-12-11 09:03:53,021] Trial 3 finished with value: 0.8906840481225947 and parameters: {'alpha': 190.41206588514927}. Best is trial 2 with value: 0.8924522414450646.
[I 2024-12-11 09:03:53,475] Trial 4 finished with value: 0.8906493776652914 and parameters: {'alpha': 13.663804553374645}. Best is trial 2 with value: 0.8924522414450646.
[I 2024-12-11 09:03:53,482] Trial 5 finished with value: 0.8907533890

In [16]:
# Mejor hiperparámetro
params = study.best_params
print(params)

{'alpha': 9933.355154619714}


In [17]:
# Mejor metrica
score = study.best_value
print(score)

0.9067711403113408


**Universidad Nacional de Colombia** - *Facultad de Ingeniería*